# Empresas Sancionadas Contratadoras

In [1]:
!pip install geopandas

You should consider upgrading via the '/Users/semigoso/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
import re
import os
import numpy as np
import pandas as pd
import json
from os import listdir
from os.path import isfile, join
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [4]:
plt.rcParams.update({'font.size': 16})

def juntar_fechas(x):
    return [x[0], x[1]]

def get_contratando_con_sanciones(x):
    
    global time_sancion_dict
    fecha_sancion = datetime.strptime(x[1], '%Y-%m-%d').date()
    
    # get periodo de sanciones:
    try:
        periodos_inhabilitado = time_sancion_dict.get(x[0])
        #print(periodos_inhabilitado)
    except:
        # Si no hay periodo de inhabilitación
        return "Falta fechas"
    # revisar si el contrato fue subscrito mientras tenia sanciones
    if periodos_inhabilitado is not None:
        for i in range(0,len(periodos_inhabilitado),2):
            ini = datetime.strptime(periodos_inhabilitado[i], '%Y-%m-%d').date()
            fin = datetime.strptime(periodos_inhabilitado[i +1], '%Y-%m-%d').date()
            if ini<=fecha_sancion<=fin:
                return True 

    return False        
    
def restar_meses(x):
    if int(x[1]) >= int(x[0]):
        year_2 , year_1 = x[1][:4] , x[0][:4]
        mes_2 , mes_1 = x[1][4:] , x[0][4:]
        date_2 , date_1 = datetime.strptime(year_2 + ':' + mes_2, '%Y:%m'), datetime.strptime(year_1 + ':' + mes_1, '%Y:%m')
        num_months = (date_2.year - date_1.year) * 12 + (date_2.month - date_1.month)
        return int(num_months)

def get_fecha_despues_sancion(x):
    global fecha_primera_sancion_dict
    
    fecha_ini_sancion = fecha_primera_sancion_dict.get(int(x[0]))
    if fecha_ini_sancion is not None:
        if datetime.strptime(x[1], '%Y-%m-%d').date()>=fecha_ini_sancion:
            return True
    else:
        return False

def formato_departamentos(x):
    if x == "PROV. CONST. DEL CALLAO":
        return "Callao"
    return x[0].upper() + x[1:].lower()

In [5]:
clasificacion_motivos = {"a desistirse o retirar injustificadamente su oferta":"A",
"desistirse o retirar injustificadamente su propuesta":"A",
"f ocasionar que la entidad resuelva el contrato incluido acuerdos marco siempre que dicha resolucion haya quedado consentida o firme en via conciliatoria o arbitral":"B",
"no mantener oferta hasta suscripcion contrato":"A",
"no mantener oferta no suscripcion injustificada del contrato no recepcion de orden de compra servicios numeral 1 del articulo 294":"A",
"no mantener su oferta hasta el consentimiento de la buena pro":"A",
"no suscribir injustificadamente el contrato o acuerdo de convenio marco":"A",
"no suscripcion de contrato":"A",
"no mantener oferta no suscripcion injustificada del contrato acuerdo no recepcion de orden de compra servicios literal a numeral 51 1 art 51":"A",
"no mantener oferta no suscripcion injustificada del contrato no recepcion de orden de compra servicios literal a numeral 51 1 art 51":"A",
"participar en practicas restrictivas de la libre competencia socios comunes no permitidos literal h numeral 51 1 art 51":"A",
"dar lugar a la resolucion del contrato orden de compra o de servicios por causal atribuible a su parte":"B",
"dar lugar a la resolucion del contrato orden de compra servicios literal b numeral 51 1 art 51":"B",
"dar lugar a la resolucion del contrato orden de compra servicios numeral 2 del articulo 294":"B",
"den lugar a la resolucion del contrato o de servicios por causal atribuible a su parte literal b numeral 51 1 art 51":"B",
"ocasionar que la entidad resuelva el contrato incluido acuerdos marco siempre que dicha resolucion haya quedado consentida o firme en via conciliatoria o arbitral":"B",
"ocasionar que la entidad resuelva el contrato siempre que dicha resolucion haya quedado consentida o firme en via conciliatoria o arbitral":"B",
"resolucion de contrato":"B",
"c contratar con el estado estando impedido conforme a ley":"C",
"contratado impedidos para contratar ley 26850 art 9":"C",
"contratar con el estado estando en cualquiera de los supuestos de impedimento previstos en el art 11 de esta ley":"C",
"contratar con el estado estando en cualquiera de los supuestos de impedimento previstos en el articulo 11 de esta ley":"C",
"contratar con el estado estando impedido para ello de acuerdo a la lce":"C",
"contratar estando impedido para ello literal d numeral 51 1 art 51":"C",
"haberse registrado como participante presentar propuestas o suscribir un contrato o acuerdo de convenio marco sin contar con inscripcion vigente en el rnp":"C",
"k suscribir contratos o acuerdos marco sin contar con inscripcion vigente en el registro nacional de proveedores rnp o suscribir contratos por montos mayores a su capacidad libre de contratacion en especialidades o categorias distintas a las autorizadas por el registro nacional de proveedores rnp":"C",
"no se encuentra inscrito en el r n c":"C",
"participar en proceso de seleccion suscribir contrato sin contar con rnp vigente numeral 5 del articulo 294":"C",
"participar suscribir contrato sin contar con rnp vigente literal e numeral 51 1 art 51":"C",
"registrarse como participante presentar propuestas suscribir contrato o acuerdo sin contar con rnp vigente":"C",
"registrarse como participantes presentar propuestas o suscribir contratos o acuerdos marco sin contar con inscripcion vigente en el registro nacional de proveedores rnp o suscribir contratos por montos mayores a su capacidad libre de contratacion o en especialidades distintas a las autorizadas por el registro nacional de proveedores rnp":"C",
"se registren como participantes presenten propuestas o suscribir contrato o acuerdos marco sin contar con inscripcion vigente en el registro nacional de proveedores rnp o suscriban contrato por montos mayores a su capacidad libre de contratacion o en especialidades distintas a las autorizadas por el rnp":"C",
"suscripcion de contratos sin estar inscritos en rnc":"C",
"suscribir contrato por montos mayores a capacidad o por especialidad distinta a la otorgada por rnp literal f numeral 51 1 art 51":"C",
"suscribir un contrato en el caso de ejecucion o consultoria de obras por montos mayores a su capacidad libre de contratacion o en especialidades distintas":"C",
"transgresion de la prohibicion del articulo 11 socios comunes no permitidos":"C",
"no proceder al saneamiento de los vicios ocultos en la prestacion a su cargo segun lo requerido por la entidad cuya existencia haya sido reconocida por el contratista o declarada en via arbitral":"D",
"negarse injustificadamente a cumplir las obligaciones derivadas del contrato cuando estas deban verificarse con posterioridad al pago o cuando el pago ya se hubiera efectuado":"D",
"h negarse injustificadamente a cumplir las obligaciones derivadas del contrato que deben ejecutarse con posterioridad al pago":"D",
"incumplimiento de obligaciones contractuales":"D",
"incumplimiento del contrato":"D",
"g no proceder al saneamiento de los vicios ocultos en la prestacion a su cargo segun lo requerido por la entidad cuya existencia haya sido reconocida por el contratista o declarada en via arbitral":"D",
"b incumplir injustificadamente con su obligacion de perfeccionar el contrato o de formalizar acuerdos marco":"D",
"incumplir con su obligacion de perfeccionar el contrato o de formalizar acuerdos marco":"D",
"incumplir injustificadamente con su obligacion de perfeccionar el contrato o de formalizar acuerdos marco":"D",
"incumplir injustificadamente las obligaciones del contrato hasta los plazos de responsabilidad establecidos en las bases":"D",
"entregar bien prestar servicio ejecutar obra con vicios ocultos literal c numeral 51 1 art 51":"D",
"declaracion jurada con informacion inexacta":"E",
"documentos falsos":"E",
"i presentar informacion inexacta a las entidades al tribunal de contrataciones del estado al registro nacional de proveedores rnp al organismo supervisor de las contrataciones del estado osce y a la central de compras publicas–peru compras":"E",
"i presentar informacion inexacta a las entidades al tribunal de contrataciones del estado o al registro nacional de proveedores rnp al organismo supervisor de las contrataciones del estado osce y a la central de compras publicas peru compras en el caso de las entidades siempre que este relacionada con el cumplimiento de un requerimiento o factor de evaluacion o requisitos que le represente una ventaja o beneficio en el procedimiento de seleccion o en la ejecucion contractual":"E",
"j presentar documentos falsos o adulterados a las entidades al tribunal de contrataciones del estado al registro nacional de proveedores rnp al organismo supervisor de las contrataciones del estado osce o a la central de compras publicas peru compras":"E",
"j presentar documentos falsos o adulterados a las entidades al tribunal de contrataciones del estado al registro nacional de proveedores rnp al organismo supervisor de las contrataciones del estado osce o a la central de compras publicas– peru compras":"E",
"presentacion de documentacion falsa y o inexacta a las entidades tribunal osce":"E",
"presentacion de documentacion falsa y o inexacta a las entidades y o osce antes consucode numeral 9 del articulo 294":"E",
"presentar documentos falsos o adulterados a las entidades al tribunal de contrataciones del estado o al registro nacional de proveedores rnp":"E",
"presentar documentos falsos o informacion inexacta a las entidades al tribunal de contrataciones del estado o al osce":"E",
"presentar informacion inexacta a las entidades al tribunal de contrataciones del estado o al registro nacional de proveedores rnp siempre que este relacionada con el cumplimiento de un requerimiento o factor de evaluacion que le represente una ventaja o beneficio en el procedimiento de seleccion o en la ejecucion contractual":"E",
"presentar informacion inexacta a las entidades al tribunal de contrataciones del estado o al registro nacional de proveedores rnp siempre que este relacionada con el cumplimiento de un requisito o con la obtencion de un beneficio o ventaja para si o para terceros":"E",
"d subcontratar prestaciones sin autorizacion de la entidad o en porcentaje mayor al permitido por el reglamento o cuando el subcontratista no cuente con inscripcion vigente en el registro nacional de proveedores rnp este impedido o inhabilitado o suspendido para contratar con el estado":"F",
"realicen subcontrataciones sin autorizacion de la entidad o por un porcentaje mayor a lo permitido literal g numeral 51 1 art 51":"F",
"realicen subcontrataciones sin autorizacion de la entidad o por un porcentaje mayor a lo permitido literal h numeral 51 1 art 51":"F",
"subcontratar prestaciones sin autorizacion de la entidad o en porcentaje mayor al permitido por el reglamento o cuando el subcontratista no cuente con inscripcion vigente en el registro nacional de proveedores rnp este impedido o inhabilitado para contratar con el estado":"F",
"subcontratar prestaciones sin autorizacion de la entidad o en porcentaje mayor al permitido por esta ley y su reglamento o cuando el subcontratista no cuente con inscripcion vigente en el registro nacional de proveedores rnp este impedido o inhabilitado para contratar con el estado":"F",
"e incumplir la obligacion de prestar servicios a tiempo completo como residente o supervisor de obra salvo en aquellos casos en que la normativa lo permita":"G",
"interponer recursos impugnativos contra actos inimpugnables literal j numeral 51 1 art 51":"G",
"interponer recursos impugnativos contra actos inimpugnables literal k numeral 51 1 art 51":"G",
"otros":"G","no definido":"G",
"perfeccionar el contrato luego de notificada en el sistema electronico de contrataciones del estado seace la suspension recomendacion de nulidad o nulidad del proceso de contratacion dispuesta por el organismo supervisor de las contrataciones del estado osce en el ejercicio de sus funciones":"G",
"se constate luego de otorgada la conformidad incumplimiento de obligaciones contractuales literal k numeral 51 1 art 51":"G",
"se constate luego de otorgada la conformidad incumplimiento de obligaciones contractuales literal l numeral 51 1 art 51":"G"
}

motivos_resumidos = {
    "A":"Interferir en el proceso correcto de contratacion",
    "B":"Resolución del contrato",
    "C":"Contratar o presentarse a convocatoria estando impedido o sin cumplir con los requisitos",
    "D":"Incumplir contrato",
    "E":"Documentación e información falsa o inexacta",
    "F":"Subcontratar sin autorización o encontra de lo permitido",
    "G":"Otros"
}

In [6]:
def get_empresas_sancionadas_contratadoras(motivos_resumidos = motivos_resumidos, 
                               clasificacion_motivos = clasificacion_motivos ):
    df_sanc = pd.read_csv(os.getcwd() + "/producto/proveedores_perfil_sanciones.csv")
    df_sanc.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df_sanc['motivos_resumidos'] = df_sanc['motivos'].apply(lambda x: clasificacion_motivos.get(x))
    print(df_sanc['motivos_resumidos'].value_counts())

    # Empresas sancionadas con info de categoria de sancion
    empresas_sancionadas = pd.pivot_table(df_sanc, index = ["ruc", "motivos_resumidos"], values="vigente", aggfunc="count").unstack()
    empresas_sancionadas.fillna(0, inplace = True)
    empresas_sancionadas['count'] = empresas_sancionadas['vigente']['A'] + empresas_sancionadas['vigente']['B'] + empresas_sancionadas['vigente']['C'] +\
                                    empresas_sancionadas['vigente']['D'] + empresas_sancionadas['vigente']['E'] + empresas_sancionadas['vigente']['F'] +\
                                        empresas_sancionadas['vigente']['G']
    empresas_sancionadas.sort_values(by=['count'], ascending=False, inplace= True)
    empresas_sancionadas.rename(columns={"vigente":"categoria"}, inplace= True)

    # Obtener informacion de contratos despues de sancion
    seace = pd.read_csv(os.getcwd() +"/producto/proveedores_perfil_seace.csv", lineterminator = '\n')
    seace.drop(['Unnamed: 0'], axis = 1, inplace = True)
    seace.dropna(subset = ['fecha'], inplace = True)
    df_sanc['fechaIni_'] = df_sanc['fechaIni_'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
    fecha_primera_sancion_df = pd.pivot_table(df_sanc, index = 'ruc', values= 'fechaIni_', aggfunc= np.min)
    global fecha_primera_sancion_dict
    fecha_primera_sancion_dict = dict(zip(fecha_primera_sancion_df.index, fecha_primera_sancion_df.fechaIni_))
    seace['despues_sancion'] = seace[['ruc', 'fecha']].apply(get_fecha_despues_sancion, axis = 1)
    contratos_despues_sancion = seace[seace['despues_sancion']==True]
    acumulado_despues_sancion = pd.pivot_table(contratos_despues_sancion, index = 'ruc', values='montoOrigen', aggfunc= sum)
    n_contratos_despues_sancion = pd.pivot_table(contratos_despues_sancion, index = 'ruc', values='despues_sancion', aggfunc= "count")

    # Agregar razon social a los rucs
    razon_social_rucs = df_sanc[df_sanc['ruc'].isin(list(empresas_sancionadas.index))][['ruc', 'Razon_social']].drop_duplicates()
    razon_social_rucs.set_index('ruc', inplace= True)

    # Concatena todo los datos
    info_empresas_sancionadas = pd.concat([razon_social_rucs,empresas_sancionadas,fecha_primera_sancion_df, \
                                        acumulado_despues_sancion, n_contratos_despues_sancion],axis= 1)
    info_empresas_sancionadas.fillna(0, inplace= True)
    info_empresas_sancionadas['montoOrigen'] = info_empresas_sancionadas['montoOrigen'].apply(lambda x:int(x))
    info_empresas_sancionadas.rename(columns={('categoria', 'A'):"A",('categoria', 'B'):"B",('categoria', 'C'):"C", ('categoria', 'D'):"D", \
                                    ('categoria', 'E'):"E", ('categoria', 'F'):"F", ('categoria', 'G'):"G",('count', ''):"sanciones", \
                                        "fechaIni_":"primera_sancion", 'montoOrigen':"monto_ganado", "despues_sancion":"contratos_ganados"}, inplace= True)

    info_empresas_sancionadas.sort_values(by = ['monto_ganado', 'sanciones'], ascending=[False, False], inplace = True)

    # Agregar info geografica
    df_geo = pd.read_csv(os.getcwd() + "/producto/proveedores_perfil_info_geo.csv")
    df_geo.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df_geo = df_geo[['ruc','departamento']]
    df_geo = df_geo[df_geo['ruc'].isin(list(info_empresas_sancionadas.index))]
    df_geo['ruc'] = df_geo['ruc'].apply(lambda x: int(x))
    df_geo.set_index('ruc', inplace= True)

    info_empresas_sancionadas= pd.concat([info_empresas_sancionadas, df_geo], axis=1)
    info_empresas_sancionadas.to_csv(os.getcwd() + '/producto/100_empresas_sancionadas_contratadoras.csv', encoding= 'utf-8-sig')

    return info_empresas_sancionadas

In [7]:
#info_empresas_sancionadas
def graficos_sanciones(info_empresas_sancionadas):
    regiones = gpd.read_file(os.getcwd() + '/geo/per_admbnda_adm1_ign_20200714.shp') 
    regiones = regiones[['ADM1_ES', 'geometry']]
    regiones['ADM1_ES'] = regiones['ADM1_ES'].apply(lambda x: formato_departamentos(x))
    regiones.set_index('ADM1_ES', inplace=True)
    regiones.index.name = "departamento"


    regiones_sanciones = pd.pivot_table(info_empresas_sancionadas.dropna(subset=['departamento']), index= 'departamento', values=["sanciones",\
                        "A","B","C","D","E","F","G"], aggfunc=sum)
    regiones_sanciones.index = list(map(formato_departamentos ,list(regiones_sanciones.index)))

    datos_geograficos = pd.concat([regiones,regiones_sanciones], axis = 1)

    # Primer plot
    fig, ax = plt.subplots(1, 1, figsize = (10,12))
    
    # Control del título y los ejes
    ax.set_title('Sanciones totales por departamento', 
                pad = 20, 
                fontdict={'fontsize':20, 'color': '#4873ab', 'weight':'bold'})
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')

    # Mostrar el mapa finalizado
    datos_geograficos.plot(cmap = 'Oranges', norm = colors.LogNorm(vmin=datos_geograficos['sanciones'].min(), vmax=datos_geograficos['sanciones'].max()), \
        column='sanciones', ax=ax, zorder=5, legend=True)

    plt.tight_layout()
    plt.savefig(os.getcwd() + "/imagenes/Sanciones totales por departamento.jpg")
    plt.close()

    for columna in datos_geograficos.columns[1:8]:
        fig, ax = plt.subplots(1, 1, figsize = (10,12))
        
        # Control del título y los ejes
        ax.set_title('Sanciones totales por departamento en categoria:\n' + motivos_resumidos.get(columna), 
                    pad = 20, 
                    fontdict={'fontsize':20, 'color': '#4873ab'})
        ax.set_xlabel('Longitud')
        ax.set_ylabel('Latitud')
        if (datos_geograficos[columna] + 1).max() >= 100:
        # Mostrar el mapa finalizado
            datos_geograficos.plot(cmap = 'Oranges', norm = colors.LogNorm(vmin=(datos_geograficos[columna] + 1).min(), vmax=(datos_geograficos[columna] + 1).max()), \
                column=datos_geograficos[columna], ax=ax, zorder=5, legend=True)
            plt.tight_layout()
            plt.savefig(os.getcwd() + "/imagenes/Sanciones totales por departamento_"+ columna + ".jpg")
            plt.close()
        else:
            datos_geograficos.plot(cmap = 'Oranges', \
                column=datos_geograficos[columna], ax=ax, zorder=5, legend=True)
            plt.tight_layout()
            plt.savefig(os.getcwd() + "/imagenes/Sanciones totales por departamento_"+ columna +".jpg")
            plt.close()

In [8]:
def get_inhabilidatos_contratando():   
    global time_sancion_dict
    df_sanc = pd.read_csv(os.getcwd() + "/producto/proveedores_perfil_sanciones.csv")
    df_sanc.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df_sanc['fechaFin_'] = df_sanc['fechaFin_'].fillna('2100-01-01')
    df_sanc['meses'] = df_sanc[['fechaIni_', 'fechaFin_']].apply(juntar_fechas, axis = 1)
    df_sanc = df_sanc[df_sanc['descripcion'] != 'MULTA']

    periodos_sancionados = pd.pivot_table(df_sanc, index = ['ruc'], values= 'meses', aggfunc= np.sum)
    time_sancion_dict = dict(zip(periodos_sancionados.index, periodos_sancionados['meses'].values))

    seace = pd.read_csv(os.getcwd() +"/producto/proveedores_perfil_seace.csv", lineterminator = '\n')
    seace.drop(['Unnamed: 0'], axis = 1, inplace = True)

    #    with open('contratos_fechas.json', 'r', encoding='utf-8') as f:
    #        contratos_fechas = json.load(f, encoding='utf-8')
    seace.dropna(subset = ['fecha'], inplace= True)
    seace['sancionado'] = seace[['ruc', 'fecha']].apply(get_contratando_con_sanciones, axis = 1)
    #seace['periodo_sancion'] = seace['ruc'].apply(lambda x: time_sancion_dict.get(int(x)))
    contratos_sancionados_c = seace[seace['sancionado'] == True]
    contratos_sancionados = contratos_sancionados_c.copy()
    contratos_sancionados.to_csv(os.getcwd() + "/producto/91_contratos_empresas_inhabilitadas.csv", encoding= 'utf-8-sig')

    top_empresas_inhabilitadas = pd.pivot_table(contratos_sancionados, index = ['ruc', 'Razon_social'], values = 'sancionado', aggfunc= 'count').sort_values('sancionado', ascending=False)
    top_empresas_inhabilitadas.rename(columns= {"sancionado":"#_contratos_durante_sancion"}, inplace = True)
    top_empresas_inhabilitadas.to_csv(os.getcwd() + "/producto/90_contratos_durante_sancion.csv", encoding= 'utf-8-sig')

In [4]:
auxpd = pd.read_csv(os.getcwd() +"/producto/planilla_buscador.csv", lineterminator = '\n',low_memory=False)


In [9]:
def obtener_files():
    directorio = os.getcwd()
    directorio_files = directorio + '/producto'
    onlyfiles = [f for f in listdir(directorio_files) if isfile(join(directorio_files, f))]
    return onlyfiles

In [10]:
obtener_files()

['90_contratos_durante_sancion.csv',
 'proveedores_perfil.csv',
 '91_contratos_empresas_inhabilitadas.csv',
 '100_empresas_sancionadas_contratadoras.csv',
 'proveedores_perfil_sanciones.csv',
 'planilla_buscador.csv',
 'proveedores_perfil_socios.csv',
 'visitantes_buscador.csv',
 'proveedores_perfil_representantes.csv',
 'proveedores_perfil_org_administrativos.csv',
 'visitantes_empresarios.csv',
 'visitantes_perfil.csv',
 'proveedores_perfil_seace.csv',
 'proveedores_buscador.csv',
 'proveedores_perfil_info_geo.csv',
 'planilla_perfil.csv']

In [11]:
def imprimir_tablas():
    for file in obtener_files():
        df = pd.read_csv(os.getcwd() + '/producto/' + file , lineterminator = '\n',low_memory=False)
        print(file, df.shape[0])

In [12]:
imprimir_tablas()

90_contratos_durante_sancion.csv 55
proveedores_perfil.csv 21293
91_contratos_empresas_inhabilitadas.csv 164
100_empresas_sancionadas_contratadoras.csv 2019
proveedores_perfil_sanciones.csv 3029
planilla_buscador.csv 1217830
proveedores_perfil_socios.csv 31867
visitantes_buscador.csv 188458
proveedores_perfil_representantes.csv 17549
proveedores_perfil_org_administrativos.csv 21701
visitantes_empresarios.csv 2006
visitantes_perfil.csv 80589
proveedores_perfil_seace.csv 2121229
proveedores_buscador.csv 21887
proveedores_perfil_info_geo.csv 23238
planilla_perfil.csv 95352


In [18]:
imprimir_tablas()

90_contratos_durante_sancion.csv 55
proveedores_perfil.csv 21293
91_contratos_empresas_inhabilitadas.csv 164
100_empresas_sancionadas_contratadoras.csv 2019
proveedores_perfil_sanciones.csv 3029
planilla_buscador.csv 1217830
proveedores_perfil_socios.csv 31867
visitantes_buscador.csv 188458
proveedores_perfil_representantes.csv 17549
proveedores_perfil_org_administrativos.csv 21701
visitantes_empresarios.csv 2006
visitantes_perfil.csv 80589
proveedores_perfil_seace.csv 2121229
proveedores_buscador.csv 21887
proveedores_perfil_info_geo.csv 23238
planilla_perfil.csv 95352


In [1]:
import pandas as pd

In [10]:
seace = pd.read_csv(os.getcwd() +"/producto/proveedores_perfil_seace.csv", lineterminator = '\n')
#seace.drop(['Unnamed: 0'], axis = 1, inplace = True)
#seace.dropna(subset = ['fecha'], inplace = True)
#seace['fecha'] = seace['fecha'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

In [13]:
seace['montoOrigen'].sum()

1258125491310.85

In [9]:
seace['montoOrigen'].sum()

178477159214.78998

In [27]:
seace.shape

(2121229, 10)

In [8]:
seace.shape
seace[seace['ruc'] == 20101266819]

,Unnamed: 0,fecha,ruc,Razon_social,nomEntCont,desContProv,montoOrigen,desEstContProv,desCatObj2,codContProv
1364948,1364948,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,JUNTA DE RESOLUCION DE DISPUTAS - HONORARIO AD...,111703.71,Contratado,SERVICIO,2@20481539
1364949,1364949,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,JUNTA DE RESOLUCION DE DISPUTAS - GASTOS ADMIN...,26982.48,Contratado,SERVICIO,2@20481538
1364950,1364950,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,PAGO DE GASTOS ADMINISTRATIVOS DE LA INSTITUCI...,189639.59,Contratado,SERVICIO,2@20481488
1364951,1364951,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,PAGO DE GASTOS ADMINISTRATIVOS DE LA INSTITUCI...,77426.81,Contratado,SERVICIO,2@20481487
1364952,1364952,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,SERVICIO NACIONAL DE METEOROLOGIA E HIDROLOGIA,PAGO DERECHO ARBITRAJE-GASTOS ADMINISTRATIVOS ...,5852.80,Contratado,SERVICIO,2@20452694
...,...,...,...,...,...,...,...,...,...,...
1366651,1366651,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MEF-AGENCIA DE PROMOCIÓN DE LA INVERSIÓN PRIVA...,CONTRATACION DE SERVICIOS DE ATENCION INTEGRA...,4352.00,Contratado,SERVICIO EN GENERAL,1@273772
1366652,1366652,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,BANCO CENTRAL DE RESERVA DEL PERÚ,"Curso Diplomado en Marketing, dos (02) partici...",5200.00,Contratado,SERVICIO EN GENERAL,1@207537
1366653,1366653,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,CONSEJO SUPERIOR DE CONTRATACIONES Y ADQUISICI...,CONTRATACION DEL SERVICIO DE ALQUILER DE LOCAL...,11462.00,Contratado,SERVICIO EN GENERAL,1@82268
1366654,1366654,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,SUPERINTENDENCIA NACIONAL DE ADMINISTRACION TR...,"CURSO DE CAPACITACION ""CALIDAD EN EL SERVICIO ...",1547.00,Contratado,SERVICIO,1@19940


In [17]:
seace = pd.read_csv(os.getcwd() +"/producto/proveedores_perfil_seace.csv", lineterminator = '\n')
seace.shape

(2119521, 10)

In [18]:
seace[seace['ruc'] == 20101266819]

,Unnamed: 0,fecha,ruc,Razon_social,nomEntCont,desContProv,montoOrigen,desEstContProv,desCatObj2,codContProv


In [33]:
socios = pd.read_csv(os.getcwd() +"/producto/proveedores_perfil_socios.csv", lineterminator = '\n')
organos = pd.read_csv(os.getcwd() +"/producto/proveedores_perfil_org_administrativos.csv", lineterminator = '\n')
representantes = pd.read_csv(os.getcwd() +"/producto/proveedores_perfil_representantes.csv", lineterminator = '\n')

In [34]:
representantes.shape

(17549, 6)

In [32]:
representantes[representantes['ruc']==20138861300]

,Unnamed: 0,ruc,Razon_social,apellidosNomb,siglaDocIde,nroDocumento
5913,5913,20138861300,ADRA PERU,martinez darmont carlos samuel,L.E.,04641932
5914,5914,20138861300,ADRA PERU,vergara serrano plinio rogelio,L.E.,25500318


In [31]:
organos[organos['ruc'] == 20138861300]

,Unnamed: 0,ruc,Razon_social,apellidosNomb,siglaDocIde,nroDocumento,descCargo,descTipoOrgano,fechaIngreso
7369,7369,20138861300,ADRA PERU,echevarria miranda david jonatan,L.E.,09761622,Vice Presidente,CONSEJO DIRECTIVO,31/07/2012
7370,7370,20138861300,ADRA PERU,martinez darmont carlos samuel,L.E.,04641932,Tesorero,CONSEJO DIRECTIVO,31/07/2012
7371,7371,20138861300,ADRA PERU,romero marin daniel,L.E.,29658141,Vocal,CONSEJO DIRECTIVO,31/07/2012
7372,7372,20138861300,ADRA PERU,torres cordova elias lucas,L.E.,16001663,Vocal,CONSEJO DIRECTIVO,31/07/2012
7373,7373,20138861300,ADRA PERU,vergara serrano plinio rogelio,L.E.,25500318,Secretario,CONSEJO DIRECTIVO,15/01/2013
7374,7374,20138861300,ADRA PERU,chavez idrogo enzo ronald,D.N.I.,08153615,Presidente,CONSEJO DIRECTIVO,07/12/2015
7375,7375,20138861300,ADRA PERU,villar espinoza daniel urias,D.N.I.,10706767,Vice Presidente,CONSEJO DIRECTIVO,17/12/2015


In [107]:
socios = socios[socios['siglaDocIde'] == 'D.N.I.']
socios.dropna(subset=['nroDocumento'], inplace = True)
dni_socios = set(list(socios['nroDocumento']))

organos = organos[organos['siglaDocIde'] == 'D.N.I.']
organos.dropna(subset=['nroDocumento'], inplace= True)
dni_organos = set(list(organos['nroDocumento']))

representantes = representantes[representantes["siglaDocIde"] == "D.N.I."]
representantes.dropna(subset=['nroDocumento'], inplace= True)
dni_representantes = set(list(representantes['nroDocumento']))

In [108]:
print(len(dni_socios))
dni_socios = dni_socios.union(dni_organos)
print(len(dni_socios))
dni_socios = dni_socios.union(dni_representantes)
print(len(dni_socios))

19212
23426
25832


In [109]:
list_socios = list(dni_socios)
list_socios = [dni_socio.strip() for dni_socio in list_socios]
list_socios = [dni_socios for dni_socios in list_socios if len(dni_socios) == 8]

In [111]:
textfile = open("dnis.txt", "w")
for element in list_socios:
    textfile.write(element + "\n")
textfile.close()

In [15]:
get_inhabilidatos_contratando()
info_empresas_sancionadas = get_empresas_sancionadas_contratadoras()
graficos_sanciones(info_empresas_sancionadas = info_empresas_sancionadas)

E    1467
B     464
D     449
A     445
C     158
G      32
F      14
Name: motivos_resumidos, dtype: int64


# Otros


In [8]:
meses_sancionados = pd.pivot_table(df_sanc, values= 'meses_sancionado', index = 'ruc', aggfunc= "count")

In [9]:
meses_sancionados.sort_values(by = "meses_sancionado", ascending= False)

,meses_sancionado
ruc,
20101266819,34
20100162076,10
20506859779,9
10421830466,9
20517623394,7
...,...
20445576060,1
20445575411,1
20445419983,1


In [10]:
df_sanc[df_sanc['ruc'] == 20101266819]

,ruc,Razon_social,descripcion,idsCausales,motivos,fechaIni_,fechaFin_,meses_sancionado,vigente,montoTexto,nroRes,fechaResol,idNumInhab
1608,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-10-25,NaN,10000,True,NaN,3460-2021-TCE-S1,2021-10-22,22255
1609,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-10-25,NaN,10000,True,NaN,3461-2021-TCE-S1,2021-10-22,22254
1610,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-11-04,NaN,10000,True,NaN,3628-2021-TCE-S1,2021-11-03,22343
1611,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-11-12,NaN,10000,True,NaN,3754-2021-TCE-S5,2021-11-11,22399
1612,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-11-16,NaN,10000,True,NaN,3788-2021-TCE-S5,2021-11-15,22415
1613,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-11-19,NaN,10000,True,NaN,3896-2021-TCE-S2,2021-11-18,22477
1614,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-11-25,NaN,10000,True,NaN,3875-2021-TCE-S2,2021-11-17,22527
1615,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-11-25,NaN,10000,True,NaN,3995-2021-TCE-S1,2021-11-24,22536
1616,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-11-25,NaN,10000,True,NaN,3999-2021-TCE-S4,2021-11-24,22535
1617,20101266819,CAMARA DE COMERCIO DE LIMA,DEFINITIVA,238,c) Contratar con el Estado estando impedido co...,2021-12-16,NaN,10000,True,NaN,4289-2021-TCE-S1,2021-12-15,22754


In [11]:
df_seace = pd.read_csv(os.getcwd() + "/producto/proveedores_perfil_seace.csv", lineterminator='\n')

In [12]:
df_seace.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [51]:
from datetime import datetime, date

In [52]:
datetime.strptime("2021-01-01", '%Y-%m-%d').date()

datetime.date(2021, 1, 1)

In [13]:
df_seace[df_seace['ruc'] == 20101266819]

,fecha,ruc,Razon_social,nomEntCont,desContProv,montoOrigen,desEstContProv,desCatObj2,codContProv
1364948,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,JUNTA DE RESOLUCION DE DISPUTAS - HONORARIO AD...,111703.71,Contratado,SERVICIO,2@20481539
1364949,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,JUNTA DE RESOLUCION DE DISPUTAS - GASTOS ADMIN...,26982.48,Contratado,SERVICIO,2@20481538
1364950,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,PAGO DE GASTOS ADMINISTRATIVOS DE LA INSTITUCI...,189639.59,Contratado,SERVICIO,2@20481488
1364951,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,MTC-PROYECTO ESPECIAL DE INFRAESTRUCTURA DE TR...,PAGO DE GASTOS ADMINISTRATIVOS DE LA INSTITUCI...,77426.81,Contratado,SERVICIO,2@20481487
1364952,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,SERVICIO NACIONAL DE METEOROLOGIA E HIDROLOGIA,PAGO DERECHO ARBITRAJE-GASTOS ADMINISTRATIVOS ...,5852.80,Contratado,SERVICIO,2@20452694
...,...,...,...,...,...,...,...,...,...
1366651,2008-05-20,20101266819,CAMARA DE COMERCIO DE LIMA,MEF-AGENCIA DE PROMOCIÓN DE LA INVERSIÓN PRIVA...,CONTRATACION DE SERVICIOS DE ATENCION INTEGRA...,4352.00,Contratado,SERVICIO EN GENERAL,1@273772
1366652,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,BANCO CENTRAL DE RESERVA DEL PERÚ,"Curso Diplomado en Marketing, dos (02) partici...",5200.00,Contratado,SERVICIO EN GENERAL,1@207537
1366653,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,CONSEJO SUPERIOR DE CONTRATACIONES Y ADQUISICI...,CONTRATACION DEL SERVICIO DE ALQUILER DE LOCAL...,11462.00,Contratado,SERVICIO EN GENERAL,1@82268
1366654,NaN,20101266819,CAMARA DE COMERCIO DE LIMA,SUPERINTENDENCIA NACIONAL DE ADMINISTRACION TR...,"CURSO DE CAPACITACION ""CALIDAD EN EL SERVICIO ...",1547.00,Contratado,SERVICIO,1@19940


In [55]:
df_seace['fecha'] = df_seace['fecha'].apply(lambda x: get_fecha(x))

In [56]:
df_seace[df_seace['fecha'] > datetime.strptime("2021-01-01", '%Y-%m-%d').date()]

,fecha,ruc,Razon_social,nomEntCont,desContProv,montoOrigen,desEstContProv,desCatObj2,codContProv
2,2021-06-01,10457862771,CALDERON HUAMAN MARISOL,GOBIERNO REGIONAL HUANCAVELICA-UNIDAD DE GESTI...,ADQUISICION DE MASCARILLA FACIAL TEXTIL DE USO...,106116.00,Contratado,BIEN,1@2057336
29,2022-08-24,10458663748,LLOCCLLA LANAZCA VERONICA,MUNICIPALIDAD PROVINCIAL DE CASTROVIRREYNA,\t CONTRATACIÓN DEL SERVICIO DE INSPECCIÓN PAR...,37050.00,Contratado,SERVICIO,1@2084066
47,2021-06-15,20600190301,SOLUCIONES INNOVADORAS J&B S.A.C.,MUNICIPALIDAD DISTRITAL DE SANTA ANA,ADQUISICIÓN DE MALLAS NYLON PARA EL PROYECTO ¿...,114000.00,Contratado,BIEN,1@2063134
48,2021-08-07,20600190301,SOLUCIONES INNOVADORAS J&B S.A.C.,MUNICIPALIDAD DISTRITAL DE COLCABAMBA - TAYACAJA,Ejecución de la obra ¿Creación del Parque Prin...,856356.99,Contratado,EJECUCIÓN DE OBRA,1@2061346
49,2021-04-21,20600190301,SOLUCIONES INNOVADORAS J&B S.A.C.,GOBIERNO REGIONAL DE AYACUCHO SEDE CENTRAL,ADQUISICIÓN DE ACERO LISO (DOWEL) INCLUYE CANA...,273830.00,Contratado,BIEN,1@2050147
...,...,...,...,...,...,...,...,...,...
2121018,2021-09-03,10266418898,GARCIA IDROGO LAURIANO,INPE-DIRECCION REGIONAL NORTE CHICLAYO,SERVICIO DE ALIMENTACIÓN PARA LOS INTERNOS(AS)...,2398050.00,Contratado,SERVICIO,1@1298633
2121020,2021-08-26,10266418898,GARCIA IDROGO LAURIANO,INPE-DIRECCION REGIONAL NORTE CHICLAYO,SERVICIO DE ALIMENTACION PARA LOS INTERNOS(AS)...,9638865.25,Contratado,SERVICIO,1@1295908
2121026,2021-08-18,10266418898,GARCIA IDROGO LAURIANO,INPE-DIRECCION REGIONAL NORTE CHICLAYO,SERVICIO DE ALIMENTACION PARA LOS INTERNOS(AS)...,12853821.75,Contratado,SERVICIO,1@1295374
2121128,2022-06-22,20608206486,SERVICIOS GENERALES KDURAN EMPRESA INDIVIDUAL ...,MUNICIPALIDAD DISTRITAL DE PATIVILCA,EJECUCION DE OBRA: MEJORAMIENTO Y REHABILITACI...,817080.17,Contratado,EJECUCIÓN DE OBRA,1@2103907


In [33]:
def obtener_files(carpeta):
    directorio = os.getcwd()
    directorio_files = directorio + "/" + carpeta
    onlyfiles = [f for f in listdir(directorio_files) if isfile(join(directorio_files, f))]
    return onlyfiles

def read_excel_convert_to_csv():
    carpeta = "CONOSCE_CONTRATOS"
    onlyfiles = obtener_files(carpeta)
    for file in onlyfiles:
        contrato = pd.read_excel(os.getcwd() + "/" + carpeta + "/" + file, header = 1)
        contrato.index.name = 'id'
        contrato.to_csv(os.getcwd() + "/conosce_contratos_csv/" + file[:-5] + '.csv')

def obtener_contrato_fecha_suscripcion():
    files = obtener_files("conosce_contratos_csv")
    dict_contratos_fecha = {}
    for file in files:
        df_aux = pd.read_csv(os.getcwd() + "/conosce_contratos_csv/" + file, low_memory = False)
        df_aux = df_aux[['N_COD_CONTRATO', 'FECHA_SUSCRIPCION_CONTRATO']]
        df_aux.drop_duplicates(inplace= True)
        dict_contratos_fecha.update(dict(zip(df_aux.N_COD_CONTRATO, df_aux.FECHA_SUSCRIPCION_CONTRATO)))

    with open('contratos_fechas.json', 'w') as outfile:
        json.dump(dict_contratos_fecha, outfile, indent= 1)

In [34]:
read_excel_convert_to_csv()
obtener_contrato_fecha_suscripcion()